In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.models import load_model

import shutil
import codecs
import numpy as np
from glob import glob
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from PIL import Image
import tensorflow as tf
from keras import backend
from keras.optimizers import adam
import pandas as pd

backend.set_image_data_format('channels_last')

In [11]:
import keras
_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None


def get_submodules_from_kwargs(kwargs):
    backend = keras.backend
    layers = keras.backend
    models = keras.models
    keras_utils = keras.utils

    return backend, layers, models, keras_utils

def _preprocess_numpy_input(x, data_format, mode, **kwargs):
    backend, _, _, _ = get_submodules_from_kwargs(kwargs)
    if not issubclass(x.dtype.type, np.floating):
        x = x.astype(backend.floatx(), copy=False)

    if mode == 'tf':
        x /= 127.5
        x -= 1.
        # print("tf ok")
        return x

    if mode == 'torch':
        x /= 255.
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

    else:
        if data_format == 'channels_first':
            # 'RGB'->'BGR'
            if x.ndim == 3:
                x = x[::-1, ...]
            else:
                x = x[:, ::-1, ...]
        else:
            # 'RGB'->'BGR'
            x = x[..., ::-1]
        mean = [103.939, 116.779, 123.68]
        std = None

    # Zero-center by mean pixel
    if data_format == 'channels_first':
        if x.ndim == 3:
            x[0, :, :] -= mean[0]
            x[1, :, :] -= mean[1]
            x[2, :, :] -= mean[2]
            if std is not None:
                x[0, :, :] /= std[0]
                x[1, :, :] /= std[1]
                x[2, :, :] /= std[2]
        else:
            x[:, 0, :, :] -= mean[0]
            x[:, 1, :, :] -= mean[1]
            x[:, 2, :, :] -= mean[2]
            if std is not None:
                x[:, 0, :, :] /= std[0]
                x[:, 1, :, :] /= std[1]
                x[:, 2, :, :] /= std[2]
    else:
        print("OK")
        x[..., 0] -= mean[0]
        x[..., 1] -= mean[1]
        x[..., 2] -= mean[2]
        if std is not None:
            x[..., 0] /= std[0]
            x[..., 1] /= std[1]
            x[..., 2] /= std[2]
    return x


def _preprocess_symbolic_input(x, data_format, mode, **kwargs):
    backend, _, _, _ = get_submodules_from_kwargs(kwargs)

    if mode == 'tf':
        x /= 127.5
        x -= 1.
        return x

    if mode == 'torch':
        x /= 255.
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        # print("torch")
    else:
        if data_format == 'channels_first':
            # 'RGB'->'BGR'
            if backend.ndim(x) == 3:
                x = x[::-1, ...]
            else:
                x = x[:, ::-1, ...]
        else:
            # 'RGB'->'BGR'
            x = x[..., ::-1]
        mean = [103.939, 116.779, 123.68]
        std = None

    mean_tensor = backend.constant(-np.array(mean))

    # Zero-center by mean pixel
    if backend.dtype(x) != backend.dtype(mean_tensor):
        x = backend.bias_add(
            x, backend.cast(mean_tensor, backend.dtype(x)),
            data_format=data_format)
    else:
        x = backend.bias_add(x, mean_tensor, data_format)
    if std is not None:
        x /= std
    return x


def preprocess_input(x, data_format=None, mode='caffe', **kwargs):
    backend, _, _, _ = get_submodules_from_kwargs(kwargs)

    if data_format is None:
        data_format = backend.image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format ' + str(data_format))

    if isinstance(x, np.ndarray):
        return _preprocess_numpy_input(x, data_format=data_format,
                                       mode=mode, **kwargs)
    else:
        return _preprocess_symbolic_input(x, data_format=data_format,
                                          mode=mode, **kwargs)

In [12]:
def center_img(img, size=None, fill_value=255):

    h, w = img.shape[:2]
    if size is None:
        size = max(h, w)
    shape = (size, size) + img.shape[2:]
    background = np.full(shape, fill_value, np.uint8)
    center_x = (size - w) // 2
    center_y = (size - h) // 2
    background[center_y:center_y + h, center_x:center_x + w] = img
    return background


def preprocess_img(img_path, img_size):

    img = Image.open(img_path)
    resize_scale = img_size / max(img.size[:2])
    img = img.resize((int(img.size[0] * resize_scale), int(img.size[1] * resize_scale)))
    img = img.convert('RGB')
    img = np.array(img)
    img = img[:, :, ::-1]
    img = center_img(img, img_size)
    return img

In [13]:
test_data_url = '../submit_example.csv'
test_df = pd.read_csv(test_data_url)
test_dir =[]
img_paths = []
for i in range(4260):
    img_paths.append(test_df['FileName'].iloc[i].split('/')[-1])

In [14]:

def preprocess_input_to(x):
    return preprocess_input(x, mode='torch')

def preprocess_input_tf(x):
    return preprocess_input(x, mode='tf')

def preprocess_input_x(x):
    return x

local_path = '../Test/'
test_data = []
for i in range(len(img_paths)):
    test_data.append(preprocess_img(os.path.join(local_path, img_paths[i]), 380))
print('total img: %d ' % (len(img_paths)))
test_data = np.array(test_data)
test_data_to = preprocess_input_to(test_data)
test_data_tf= preprocess_input_tf(test_data)
test_data_x = preprocess_input_x(test_data)

total img: 4260 
OK


In [16]:
print(len(test_data_tf),
len(test_data_to),
len(test_data_x),)

4260 4260 4260


In [28]:
from keras import backend
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard, Callback
from keras.layers import Dense, GlobalAveragePooling2D,Dropout
from keras.models import Model
import efficientnet.keras as efn
from keras.optimizers import Adam, RMSprop
def model_fn(objective, optimizer, metrics):
    base_model = efn.EfficientNetB4(include_top=False,
                                    input_shape=(380, 380, 3),
                                    classes=9, )
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
    return model
optimizer = adam(lr=0.01, clipnorm=0.001)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model1 = model_fn(objective, optimizer, metrics)
model1.load_weights('./model_snapshots/S0011/weights_039_0.8672.h5')
print("load ok!")

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
load ok!


In [23]:
from keras.layers import Dense, GlobalAveragePooling2D,Dropout
from keras.models import Model
from keras.optimizers import Adam
from models.mobilenet_v2 import MobileNetV2
from Groupnormalization import GroupNormalization
def model_fn(objective, optimizer, metrics):
    base_model = MobileNetV2(include_top=False,
                                                          alpha=1.4,
                                    input_shape=(380, 380, 3),
                                    classes=9 )
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
#     model.summary( )
    return model
optimizer = Adam(lr=0.01, clipnorm=0.001)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model2 = model_fn(objective, optimizer, metrics)
model2.load_weights('./model_snapshots/S0015/weights_038_0.8438.h5')
print("load ok!")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 380, 380, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 381, 381, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 190, 190, 48) 1296        Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 190, 190, 48) 192         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [24]:
from models.densenet import DenseNet201
from Groupnormalization import GroupNormalization
def model_fn(objective, optimizer, metrics):
    base_model = DenseNet201(include_top=False,
                                    input_shape=(380, 380, 3),
                                    classes=9 )
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
#     model.summary( )
    return model
optimizer = adam(lr=0.01, clipnorm=0.001)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model3 = model_fn(objective, optimizer, metrics)
model3.load_weights('./model_snapshots/S0014/weights_037_0.8688.h5')
print("load ok!")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 380, 380, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 386, 386, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 190, 190, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 190, 190, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [38]:
from models.xception import Xception
from Groupnormalization import GroupNormalization
def model_fn(objective, optimizer, metrics):
    base_model = Xception(include_top=False,
                                    input_shape=(380, 380, 3),
                                    classes=9 )
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
#     model.summary( )
    return model
optimizer = adam(lr=0.01, clipnorm=0.001)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model4 = model_fn(objective, optimizer, metrics)
model4.load_weights('./model_snapshots/S0016/weights_031_0.8578.h5')
print("load ok!")

load ok!


In [26]:
from models.inception_v3 import InceptionV3,preprocess_input
from Groupnormalization import GroupNormalization
def model_fn(objective, optimizer, metrics):
    base_model = InceptionV3(include_top=False,
                                    input_shape=(380,380, 3),
                                    classes=9 )
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
#     model.summary( )
    return model
optimizer = adam(lr=0.01, clipnorm=0.001)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model5= model_fn(objective, optimizer, metrics)
model5.load_weights('./model_snapshots/S0013/weights_038_0.8547.h5')
print("load ok !")

load ok !


In [29]:
from classification_models.keras import Classifiers
resnet152, preprocess_input = Classifiers.get('resnet152')
def model_fn(objective, optimizer, metrics):
    base_model = resnet152(include_top=False,
                                    input_shape=(380,380, 3),
                                    classes=9 )
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
#     model.summary( )
    return model
optimizer = adam(lr=0.01, clipnorm=0.001)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model6= model_fn(objective, optimizer, metrics)
model6.load_weights('./model_snapshots/S0017/weights_049_0.8547.h5')
print("load ok !")

load ok !


In [31]:
from classification_models.keras import Classifiers
resnext50, preprocess_input = Classifiers.get('resnext50')
def model_fn(objective, optimizer, metrics):
    base_model = resnext50(include_top=False,
                                    input_shape=(380,380, 3),
                                    classes=9 )
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
#     model.summary( )
    return model
optimizer = adam(lr=0.01, clipnorm=0.001)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model7= model_fn(objective, optimizer, metrics)
model7.load_weights('./model_snapshots/S0021/weights_047_0.8531.h5')
print("load ok !")

load ok !


In [39]:
model =[]
model.append(model1)
model.append(model2)
model.append(model3)
model.append(model4)
model.append(model5)
model.append(model6)
model.append(model7)
model

In [33]:
# eff DenseNet
#     return _preprocess_input(x, mode='torch', **kwargs)
# Inceptionv3 MobileNetv2 Xception
# return imagenet_utils.preprocess_input(x, mode='tf', **kwargs)
# resnet152 resnext101
# def preprocess_input(x, **kwargs):
label = []
predicts1 = np.argmax(model[0].predict(test_data_to), axis=1)
label.append(predicts1)
infos = []
for index, pred_ in enumerate(predicts1):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model1.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [34]:
predicts2 = np.argmax(model[1].predict(test_data_tf), axis=1)
label.append(predicts2)
infos = []
for index, pred_ in enumerate(predicts2):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model2.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [36]:
predicts3 = np.argmax(model[2].predict(test_data_to), axis=1)
label.append(predicts3)
infos = []
for index, pred_ in enumerate(predicts3):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model3.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [40]:
predicts4 = np.argmax(model[3].predict(test_data_tf), axis=1)
label.append(predicts4)
infos = []
for index, pred_ in enumerate(predicts4):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model4.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [41]:
predicts5 = np.argmax(model[4].predict(test_data_tf), axis=1)
label.append(predicts5)
infos = []
for index, pred_ in enumerate(predicts5):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model5.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [42]:
predicts6 = np.argmax(model[5].predict(test_data_x), axis=1)
label.append(predicts6)
infos = []
for index, pred_ in enumerate(predicts6):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model6.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [43]:
predicts7 = np.argmax(model[6].predict(test_data_x), axis=1)
label.append(predicts7)
infos = []
for index, pred_ in enumerate(predicts7):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model7.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [44]:
pred = label

In [45]:
import scipy.stats as ss
pred = np.array(pred)
pred = np.transpose(pred, (1, 0))
pred = ss.mode(pred, axis=1)[0]
predictions = np.squeeze(pred)
predictions

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


array([3, 2, 1, ..., 4, 2, 1])

In [46]:
infos = []
for index, pred_ in enumerate(predictions):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/accuracy_merge_5.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

end


In [47]:
#  tta
from edafa import ClassPredictor
class myPredictor(ClassPredictor):
    def __init__(self,model,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.model = model
        
    def predict_patches(self,patches):
        return self.model.predict(patches)


In [48]:
p1 = myPredictor(model1,"./conf/imagenet.json")
p2 = myPredictor(model2,"./conf/imagenet.json")
p3 = myPredictor(model3,"./conf/imagenet.json")
p4 = myPredictor(model4,"./conf/imagenet.json")
p5 = myPredictor(model5,"./conf/imagenet.json")
p6 = myPredictor(model6,"./conf/imagenet.json")
p7 = myPredictor(model7,"./conf/imagenet.json")


In [49]:
preds_with = []

preds_with1 = p1.predict_images(test_data_to)
preds_with1_ = np.argmax(preds_with1, axis=1)
preds_with.append(preds_with1_)
print("1 ok")

infos = []
for index, pred_ in enumerate(preds_with1_):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model1_tta.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

1 ok
end


In [50]:
preds_with2 = p2.predict_images(test_data_tf)
preds_with2_ = np.argmax(preds_with2, axis=1)
preds_with.append(preds_with2_)
print("2 ok")
infos = []
for index, pred_ in enumerate(preds_with2_):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model2_tta.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

2 ok
end


In [51]:
preds_with3 = p3.predict_images(test_data_to)
preds_with3_ = np.argmax(preds_with3, axis=1)
preds_with.append(preds_with3_)
print("3 ok")
infos = []
for index, pred_ in enumerate(preds_with3_):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model3_tta.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

3 ok
end


In [52]:
preds_with4 = p4.predict_images(test_data_tf)
preds_with4_ = np.argmax(preds_with4, axis=1)
preds_with.append(preds_with4_)
print("4 ok")
infos = []
for index, pred_ in enumerate(preds_with4_):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model4_tta.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

4 ok
end


In [53]:
preds_with5 = p5.predict_images(test_data_tf)
preds_with5_ = np.argmax(preds_with5, axis=1)
preds_with.append(preds_with5_)
print("5 ok")
infos = []
for index, pred_ in enumerate(preds_with5_):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model5_tta.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

5 ok
end


In [54]:
preds_with6 = p6.predict_images(test_data_x)
preds_with6_ = np.argmax(preds_with6, axis=1)
preds_with.append(preds_with6_)
print("6 ok")
infos = []
for index, pred_ in enumerate(preds_with6_):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model6_tta.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

6 ok
end


In [55]:
preds_with7 = p7.predict_images(test_data_x)
preds_with7_ = np.argmax(preds_with7, axis=1)
preds_with.append(preds_with7_)
print("7 ok")
infos = []
for index, pred_ in enumerate(preds_with7_):
#     pred_label = np.argmax(pred, axis=0)
    pred_label = pred_+1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
eval_weights_path='./'
result_file_name = os.path.join(os.path.dirname(eval_weights_path),
                                './merget/model7_tta.csv' )
with open(result_file_name, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

7 ok
end


In [56]:
import scipy.stats as ss
pred = np.array(preds_with)
pred = np.transpose(pred, (1, 0))
pred = ss.mode(pred, axis=1)[0]
predictions = np.squeeze(pred)
predictions

array([3, 2, 1, ..., 4, 2, 1])

In [57]:
print('###################  wirte ing  ##############################')
infos = []
for index, pred_ in enumerate(predictions):
    pred_label = pred_ + 1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
path = './merget/model_5_merget_tta.csv'
with open(path, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

###################  wirte ing  ##############################
end


In [42]:
# # print("load ok")
# # tta_model = tta_classification(model,h_flip=True, 
# #                              h_shift=(-5, 5), merge='mean')
import pandas as pd
best_sub = pd.read_csv('./result/accuracy_merge.csv')
best_acc = best_sub['type']
now_sub = pd.read_csv('./result/accuracy_merge_5.csv')
no_acc = now_sub['type']

In [43]:
num = 0
for i in range(len(no_acc)):
    if (no_acc[i]==best_acc[i]):
        num +=1
acc = num / len(no_acc)

In [44]:
acc

0.971830985915493